<div dir="rtl" class="alert alert-block alert-info"> <font size="5" face="HM XNiloofar"> 
<b> روش‌های ریاضی در مهندسی - دکتر امینی: تمرین شبیه‌سازی </b>
</font> </div>

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
<br />
مهلت تحویل تمرین: ۶ مرداد ۱۴۰۱ <br />
فقط قسمت‌های <code>TODO</code> را پر کنید و  Jupyter Notebook تکمیل شده را به فرمت <code>ipynb</code> و <code>html</code> به عنوان گزارش خود در CW آپلود کنید. <br />
ابهامات و سوالات خود در مورد این تمرین را می توانید با طراح تمرین مطرح کنید. <br />
<div dir="ltr">@hamidreza_ehteram </div> <br />  
در این تمرین شما با روش‌های عددی در تجزیه‌های ماتریسی و یادگیری واژه‌نامه به منظور نویززدایی تصاویر  آشنا می‌شوید. <br />
</font> </div>

***You can jump directly to these sections:***

+ [A. Matrix Decompositions](#A.-Matrix-Decompositions)
    + [1. Schur](#1.-Schur)
    + [2. SVD](#2.-SVD)
    + [3. Cholesky](#3.-Cholesky)
    + [4. QR](#4.-QR)
+ [B. Dictionary Learning](#B.-Dictionary-Learning)
    + [1. Create dataset](#1.-Create-dataset)
    + [2. Initialize dictionary](#2.-Initialize-dictionary)    
    + [3. Learn dictionary](#3.-Learn-dictionary) 
    + [4. Reconstruct image](#4.-Reconstruct-image)     
    + [5. Evaluation](#5.-Evaluation)     
+ [Give Us Feedback](#Give-Us-Feedback)

---

# A. Matrix Decompositions

In [1]:
# The allowed libraries
import time
import numpy as np

## 1. Schur


$$A \in \mathbb{R}^{n\times n} \text{ is a symmetric matrix} \iff \exists \text{ an orthogonal matrix } Q: Q^\top A Q = \Lambda = \text{diag}(\lambda_1,\ldots,\lambda_n)$$

<div dir="rtl"> <font size="4" face="HM XNiloofar">
الگوریتم jacobi را در حالت  cyclic پیاده‌سازی کنید.
</font> </div>

```python
d, Q = my_schur(A)
```

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
ورودی یک ماتریس متقارن حقیقی دلخواه و خروجی مقادیر ویژه و بردار‌های ویژه متناظر در تجزیه schur است. 
<br />
Cell مربوط به ارزیابی کد را اجرا کنید. خروجی الگوریتم خود را با خروجی تابع numpy مقایسه کنید. درباره‌ی تفاوت‌ها توضیح دهید.
<br />
Cell تست زمان و صحت را جهت ارزیابی کارایی الگوریتم خود اجرا کنید.
</font> </div>

In [2]:
def off_diag(A):
    result = np.linalg.norm(A)**2 - np.linalg.norm(np.diag(A))**2
    if result < 0:
        return 0
    return np.sqrt(result)

def my_schur(A):
    
    n = A.shape[0]
    Q = np.eye(n, n)
    tol = 1e-6
    delta = tol * np.linalg.norm(A)
    
    while off_diag(A) > delta:
        for p in range(n):
            for q in range(p+1, n):
                theta = 0.5 * np.arctan(2*A[p, q]/(A[q, q] - A[p, p]))
                J_theta = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
                A[[p, q], :] = J_theta.T @ A[[p, q], :]
                A[:, [p, q]] = A[:, [p, q]] @ J_theta
                Q[:, [p, q]] = Q[:, [p, q]] @ J_theta
                
    d = np.diag(A)
    return d, Q

In [3]:
# evaluate
n = 8
A = np.random.randn(n,n)
A = (A + A.transpose()) / 2

# numpy code
d, Q = np.linalg.eig(A.copy())
print(f'numpy| orthogonal test error ---> {np.linalg.norm(np.eye(n) - Q @ Q.T)}')
print(f'numpy| correctness test error ---> {np.linalg.norm(A - Q @ np.diag(d) @ Q.T)}')

# your code
my_d, my_Q = my_schur(A.copy())
print(f'yours| orthogonal test error ---> {np.linalg.norm(np.eye(n) - my_Q @ my_Q.T)}')
print(f'yours| correctness test error ---> {np.linalg.norm(A - my_Q @ np.diag(my_d) @ my_Q.T)}')

# print
print(f'A = \n{np.around(A, 3)}')
print(f'd = \n{np.around(d, 3)}')
print(f'my_d = \n{np.around(my_d, 3)}')
print(f'Q = \n{np.around(Q, 3)}')
print(f'my_Q = \n{np.around(my_Q, 3)}')

numpy| orthogonal test error ---> 2.650912686441529e-15
numpy| correctness test error ---> 5.2583746527985845e-15
yours| orthogonal test error ---> 2.0018771676235583e-15
yours| correctness test error ---> 3.267157477607925e-08
A = 
[[-0.234  0.044  1.111  0.046  0.172 -1.088 -0.328 -0.822]
 [ 0.044 -0.356  0.09   0.812  0.1   -0.602 -0.64  -0.362]
 [ 1.111  0.09  -0.708 -0.118  0.37   0.054 -0.388  0.249]
 [ 0.046  0.812 -0.118  1.252  1.21   0.023  0.87   0.131]
 [ 0.172  0.1    0.37   1.21  -0.42   0.125 -0.573 -0.057]
 [-1.088 -0.602  0.054  0.023  0.125  0.511  0.097 -0.513]
 [-0.328 -0.64  -0.388  0.87  -0.573  0.097  0.577 -0.753]
 [-0.822 -0.362  0.249  0.131 -0.057 -0.513 -0.753  0.432]]
d = 
[-2.445  2.236  1.962 -1.635  1.252  0.659 -0.013 -0.959]
my_d = 
[ 1.962 -1.635 -2.445  2.236 -0.959 -0.013  0.659  1.252]
Q = 
[[-0.529  0.051 -0.453 -0.401 -0.49   0.105  0.315  0.033]
 [-0.332  0.23  -0.291  0.311  0.009  0.027 -0.662  0.465]
 [ 0.424 -0.017 -0.306  0.37  -0.082  0.25

**Your answer: ```TODO ```**

In [4]:
# # time and correctness test
# numpy_test = {'time': 0.0, 'orthogonal test error':0.0, 'correctness test error':0.0}
# yours_test = {'time': 0.0, 'orthogonal test error':0.0, 'correctness test error':0.0}

# num_test = 100
# for i in range(num_test):
#     n = np.random.randint(40, 50)
#     A = np.random.randn(n,n)
#     A = (A + A.transpose()) / 2

#     # numpy code
#     start_time = time.time()
#     d, Q = np.linalg.eig(A.copy())
    
#     numpy_test['time'] += time.time() - start_time
#     numpy_test['orthogonal test error'] += np.linalg.norm(np.eye(n) - Q @ Q.T)
#     numpy_test['correctness test error'] += np.linalg.norm(A - Q @ np.diag(d) @ Q.T)

#     # your code
#     start_time = time.time()
#     my_d, my_Q = my_schur(A.copy())
    
#     yours_test['time'] += time.time() - start_time
#     yours_test['orthogonal test error'] += np.linalg.norm(np.eye(n) - my_Q @ my_Q.T)
#     yours_test['correctness test error'] += np.linalg.norm(A - my_Q @ np.diag(my_d) @ my_Q.T)

# for name in numpy_test:
#     numpy_test[name] /= num_test
#     yours_test[name] /= num_test

# yours_check = {'time': yours_test['time'] <= 0.5, 
#                'orthogonal test error': yours_test['orthogonal test error'] <= 1e-5, 
#                'correctness test error': yours_test['correctness test error'] <= 1e-5}

# print(f'numpy| test ---> {numpy_test}')
# print(f'yours| test ---> {yours_test}')
# print(f'yours| check ---> {yours_check}')

## 2. SVD

$$A \in \mathbb{R}^{m\times n} \text{ is a matrix} \iff \exists \text{ two orthogonal matrices } U \text{ and } V: U^\top A V = \Sigma = \text{diag}(\sigma_1,\ldots,\sigma_p), p=\min\{m,n\}, \sigma_1 \geq \ldots \geq \sigma_p \geq 0$$

<div dir="rtl"> <font size="4" face="HM XNiloofar">
الگوریتم two-sided را در حالت  cyclic پیاده‌سازی کنید.
</font> </div>

```python
U, s, VT = my_svd(A)
```

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
ورودی یک ماتریس حقیقی دلخواه و خروجی ماتریس بردارهای تکین چپ، مقادیر تکین و ترانهاده ماتریس بردارهای تکین راست است. 
<br />
<b>ملاحظه.</b> الگوریتم شما باید ماتریس‌های خروجی را به صورتی به دست آورد که مقادیر تکین مثبت و نزولی باشند
<br />
Cell مربوط به ارزیابی کد را اجرا کنید. خروجی الگوریتم خود را با خروجی تابع numpy مقایسه کنید. درباره‌ی تفاوت‌ها توضیح دهید.
<br />
Cell تست زمان و صحت را جهت ارزیابی کارایی الگوریتم خود اجرا کنید.
</font> </div>

In [5]:
def my_svd(A):
    
    m, n = A.shape
    is_wide = n > m
    if is_wide:
        A = A.T
        m, n = A.shape


    UT = np.eye(m, m)
    V = np.eye(n, n)
    tol = 1e-6
    delta = tol * np.linalg.norm(A)
    
    while off_diag(A) > delta:
        for q in range(n):
            for p in range(q+1, n):
                # Making sub-matrix symmetric first
                alpha_sym = np.arctan2((A[q, p] - A[p, q]), (A[p, p] + A[q, q]))
                J_alpha_sym = np.array([[np.cos(alpha_sym), np.sin(alpha_sym)], [-np.sin(alpha_sym), np.cos(alpha_sym)]])
                UT[[q, p], :] = J_alpha_sym.T @ UT[[q, p], :]
                A[[q, p], :] = J_alpha_sym.T @ A[[q, p], :]
                theta = 0.5 * np.arctan2(2*A[q, p], (A[p, p] - A[q, q]))
                J_theta = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
                A[[q, p], :] = J_theta.T @ A[[q, p], :]
                UT[[q, p], :] = J_theta.T @ UT[[q, p], :]
                A[:, [q, p]] = A[:, [q, p]] @ J_theta
                V[:, [q, p]] = V[:, [q, p]] @ J_theta
            for p in range(n, m):
                theta = np.arctan(-A[p, q]/A[q, q])
                J_theta = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
                A[[q, p], :] = J_theta.T @ A[[q, p], :]
                UT[[q, p], :] = J_theta.T @ UT[[q, p], :]
    
    U = UT.T
    VT = V.T
    s = np.diag(A)
    s.setflags(write=True)
    U.setflags(write=True)
    
    negative_ind = np.where(s < 0)
    s[negative_ind] = -s[negative_ind]
    U[:, negative_ind] = -U[:, negative_ind]
    
    if is_wide:
        U, s, VT = V, s.T, U.T
    
    return U, s, VT

In [6]:
# evaluate
# m, n = 8, 6
m, n = 40, 50
A = np.random.randn(m,n)

# numpy code
U, s, VT = np.linalg.svd(A.copy())
S = np.zeros((m,n)); S[[range(min(m,n))],[range(min(m,n))]] = s
print(f'numpy| orthogonal test error ---> {np.linalg.norm(np.eye(m) - U @ U.T) + np.linalg.norm(np.eye(n) - VT @ VT.T)}')
print(f'numpy| correctness test error ---> {np.linalg.norm(A - U @ S @ VT)}')

# your code
my_U, my_s, my_VT  = my_svd(A.copy())
my_S = np.zeros((m,n)); my_S[[range(min(m,n))],[range(min(m,n))]] = my_s
print(f'yours| orthogonal test error ---> {np.linalg.norm(np.eye(m) - my_U @ my_U.T) + np.linalg.norm(np.eye(n) - my_VT @ my_VT.T)}')
print(f'yours| correctness test error ---> {np.linalg.norm(A - my_U @ my_S @ my_VT)}')

# print
print(f'A = \n{np.around(A, 3)}')
print(f'U = \n{np.around(U, 3)}')
print(f'my_U = \n{np.around(my_U, 3)}')
print(f's = \n{np.around(s, 3)}')
print(f'my_s = \n{np.around(my_s, 3)}')
print(f'VT = \n{np.around(VT, 3)}')
print(f'my_VT = \n{np.around(my_VT, 3)}')

numpy| orthogonal test error ---> 1.9961402541801444e-14
numpy| correctness test error ---> 8.193065811594457e-14
yours| orthogonal test error ---> 2.8368949821855905e-14
yours| correctness test error ---> 3.329295732664009e-08
A = 
[[-0.851  0.479  0.852 ...  0.316 -0.549 -0.329]
 [-0.879 -0.48   2.481 ... -0.66  -0.672  0.676]
 [-0.466  0.423 -0.193 ...  2.346  1.215 -0.541]
 ...
 [-0.46   1.06  -1.723 ...  1.802 -1.056  1.8  ]
 [-0.026 -1.044  0.792 ... -0.518  0.471 -2.353]
 [ 0.876 -0.164 -1.834 ... -0.315 -1.875 -1.04 ]]
U = 
[[ 0.086 -0.003  0.406 ... -0.024  0.005  0.08 ]
 [-0.359 -0.127  0.211 ...  0.081 -0.349  0.184]
 [ 0.003 -0.008 -0.099 ...  0.048  0.169  0.186]
 ...
 [ 0.207 -0.351 -0.122 ...  0.096  0.053  0.144]
 [-0.116  0.092  0.116 ... -0.086  0.286 -0.039]
 [ 0.312 -0.016 -0.003 ... -0.108 -0.076  0.145]]
my_U = 
[[ 0.08  -0.005  0.024 ...  0.406 -0.003  0.086]
 [ 0.184  0.349 -0.081 ...  0.211 -0.127 -0.359]
 [ 0.186 -0.169 -0.048 ... -0.099 -0.008  0.003]
 ...
 [

**Your answer: ```TODO ```**

In [7]:
# # time and correctness test
# numpy_test = {'time': 0.0, 'orthogonal test error':0.0, 'correctness test error':0.0}
# yours_test = {'time': 0.0, 'orthogonal test error':0.0, 'correctness test error':0.0}

# num_test = 100
# for i in range(num_test):
#     m, n = np.random.randint(40, 50), np.random.randint(40, 50)
#     A = np.random.randn(m,n)

#     # numpy code
#     start_time = time.time()
#     U, s, VT = np.linalg.svd(A.copy())
#     S = np.zeros((m,n)); S[[range(min(m,n))],[range(min(m,n))]] = s
    
#     numpy_test['time'] += time.time() - start_time
#     numpy_test['orthogonal test error'] += np.linalg.norm(np.eye(m) - U @ U.T) + np.linalg.norm(np.eye(n) - VT @ VT.T)
#     numpy_test['correctness test error'] += np.linalg.norm(A - U @ S @ VT)

#     # your code
#     start_time = time.time()
#     my_U, my_s, my_VT  = my_svd(A.copy())
#     my_S = np.zeros((m,n)); my_S[[range(min(m,n))],[range(min(m,n))]] = my_s
    
#     yours_test['time'] += time.time() - start_time
#     yours_test['orthogonal test error'] += np.linalg.norm(np.eye(m) - my_U @ my_U.T) + np.linalg.norm(np.eye(n) - my_VT @ my_VT.T)
#     yours_test['correctness test error'] += np.linalg.norm(A - my_U @ my_S @ my_VT)

# for name in numpy_test:
#     numpy_test[name] /= num_test
#     yours_test[name] /= num_test

# yours_check = {'time': yours_test['time'] <= 1.0, 
#                'orthogonal test error': yours_test['orthogonal test error'] <= 1e-5, 
#                'correctness test error': yours_test['correctness test error'] <= 1e-5}

# print(f'numpy| test ---> {numpy_test}')
# print(f'yours| test ---> {yours_test}')
# print(f'yours| check ---> {yours_check}')

## 3. Cholesky

$$A \in \mathbb{R}^{n\times n} \text{ is a symmetric positive definite matrix} \iff \exists \text{ a lower-triangular matrix } G: A = G G^\top$$

<div dir="rtl"> <font size="4" face="HM XNiloofar">
به کمک تبدیل‌های گوسی الگوریتم عددی برای محاسبه تجزیه Cholesky  پیاده‌سازی کنید.
</font> </div>

```python
G = my_cholesky(A)
```

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
 ورودی یک ماتریس متقارن مثبت معین دلخواه و خروجی فاکتور Cholesky   است. 
<br />
Cell مربوط به ارزیابی کد را اجرا کنید. خروجی الگوریتم خود را با خروجی تابع numpy مقایسه کنید. درباره‌ی تفاوت‌ها توضیح دهید.
<br />
Cell تست زمان و صحت را جهت ارزیابی کارایی الگوریتم خود اجرا کنید.
</font> </div>

In [8]:
def my_cholesky(A):
    
    n = A.shape[0]
    L = np.eye(n, n)
    
    for q in range(n-1):
        t = np.zeros((n, 1))
        e = np.zeros((n, 1))
        t[q+1:n, 0] = A[q+1:n, q]/A[q, q]
        e[q, 0] = 1
        A[q+1:n, :] = A[q+1:n, :] - t[q+1:n, [0]] @ A[[q], :]
        L[:, [q]] = L @ (e + t)
    
    G = L
    for i in range(n):
        G[:, i] = G[:, i]*np.sqrt(A[i, i])    
    
    return G

In [9]:
# evaluate
n = 8
A = np.random.randn(n,n)
A = A @ np.diag(np.random.rand(n)) @ A.T

# numpy code
G = np.linalg.cholesky(A.copy())
print(f'numpy| lower-triangular test error ---> {np.linalg.norm(np.triu(G,1))}')
print(f'numpy| correctness test error ---> {np.linalg.norm(A - G @ G.T)}')

# your code
my_G = my_cholesky(A.copy())
print(f'yours| lower-triangular test error ---> {np.linalg.norm(np.triu(my_G,1))}')
print(f'yours| correctness test error ---> {np.linalg.norm(A - my_G @ my_G.T)}')

# print
print(f'A = \n{np.around(A, 3)}')
print(f'G = \n{np.around(G, 3)}')
print(f'my_G = \n{np.around(my_G, 3)}')

numpy| lower-triangular test error ---> 0.0
numpy| correctness test error ---> 2.598895994281099e-15
yours| lower-triangular test error ---> 0.0
yours| correctness test error ---> 3.997914041421348e-15
A = 
[[ 7.05  -0.677 -1.561 -0.021  2.668  0.6   -6.271 -2.146]
 [-0.677  4.002 -0.107 -2.58   0.857 -3.704  0.51   3.238]
 [-1.561 -0.107  3.648 -2.621 -0.535  0.651  2.704 -2.127]
 [-0.021 -2.58  -2.621  7.003 -0.292  2.343 -0.992  1.167]
 [ 2.668  0.857 -0.535 -0.292  2.507  0.205 -3.036  0.514]
 [ 0.6   -3.704  0.651  2.343  0.205  8.274 -6.    -3.618]
 [-6.271  0.51   2.704 -0.992 -3.036 -6.    15.795  3.121]
 [-2.146  3.238 -2.127  1.167  0.514 -3.618  3.121  6.591]]
G = 
[[ 2.655  0.     0.     0.     0.     0.     0.     0.   ]
 [-0.255  1.984  0.     0.     0.     0.     0.     0.   ]
 [-0.588 -0.129  1.813  0.     0.     0.     0.     0.   ]
 [-0.008 -1.301 -1.541  1.713  0.     0.     0.     0.   ]
 [ 1.005  0.561  0.071  0.324  1.035  0.     0.     0.   ]
 [ 0.226 -1.837  0.3

**Your answer: ```TODO ```**

In [10]:
# # time and correctness test
# numpy_test = {'time': 0.0, 'lower-triangular test error':0.0, 'correctness test error':0.0}
# yours_test = {'time': 0.0, 'lower-triangular test error':0.0, 'correctness test error':0.0}

# num_test = 100
# for i in range(num_test):
#     n = np.random.randint(40, 50)
#     A = np.random.randn(n,n)
#     A = A @ np.diag(np.random.rand(n)) @ A.T

#     # numpy code
#     start_time = time.time()
#     G = np.linalg.cholesky(A.copy())
    
#     numpy_test['time'] += time.time() - start_time
#     numpy_test['lower-triangular test error'] += np.linalg.norm(np.triu(G,1))
#     numpy_test['correctness test error'] += np.linalg.norm(A - G @ G.T)

#     # your code
#     start_time = time.time()
#     my_G = my_cholesky(A.copy())
    
#     yours_test['time'] += time.time() - start_time
#     yours_test['lower-triangular test error'] += np.linalg.norm(np.triu(my_G,1))
#     yours_test['correctness test error'] += np.linalg.norm(A - my_G @ my_G.T)

# for name in numpy_test:
#     numpy_test[name] /= num_test
#     yours_test[name] /= num_test

# yours_check = {'time': yours_test['time'] <= 0.01, 
#                'lower-triangular test error': yours_test['lower-triangular test error'] <= 1e-5, 
#                'correctness test error': yours_test['correctness test error'] <= 1e-5}

# print(f'numpy| test ---> {numpy_test}')
# print(f'yours| test ---> {yours_test}')
# print(f'yours| check ---> {yours_check}')

## 4. QR

$$A \in \mathbb{R}^{m\times n} \text{ is a matrix} \iff \exists \text{ an orthogonal matrix } Q \text{ and an upper-triangular matrix } R: A = QR$$

<div dir="rtl"> <font size="4" face="HM XNiloofar">
به کمک تبدیل‌های householder الگوریتم عددی برای محاسبه تجزیه QR  پیاده‌سازی کنید.
</font> </div>

```python
Q, R = my_qr(A)
```

<div dir="rtl"> <font size="4" face="HM XNiloofar"> 
 ورودی یک ماتریس دلخواه و خروجی ماتریس مربعی متعامد و ماتریس بالا مثلثی در تجزیه QR   است. 
<br />
Cell مربوط به ارزیابی کد را اجرا کنید. خروجی الگوریتم خود را با خروجی تابع numpy مقایسه کنید. درباره‌ی تفاوت‌ها توضیح دهید.
<br />
Cell تست زمان و صحت را جهت ارزیابی کارایی الگوریتم خود اجرا کنید.
</font> </div>

In [11]:
def my_qr(A):
    
    m, n = A.shape
    QT = np.eye(m, m)    
    
    for q in range(np.min((m, n))):
        aqq = A[q:, [q]]
        eq = np.zeros((m-q, 1))
        eq[0, 0] = 1
        v = np.zeros((m, 1))
        v[q:, [0]] = aqq + np.sign(A[q, q])*np.linalg.norm(aqq)*eq
        beta = 2/(v.T @ v)
        A = A - (beta*v) @ (v.T @ A)
        QT = QT - (beta*v) @ (v.T @ QT)
        
    R = A
    Q = QT.T
    
    return Q, R

In [12]:
# evaluate
m ,n = 8, 6
A = np.random.randn(m,n)

# numpy code
Q, R = np.linalg.qr(A.copy(), mode='complete')
print(f'numpy| upper-triangular and  orthogonal test error ---> {np.linalg.norm(np.tril(R,-1)) + np.linalg.norm(np.eye(m) - Q @ Q.T)}')
print(f'numpy| correctness test error ---> {np.linalg.norm(A - Q @ R)}')

# your code
my_Q, my_R = my_qr(A.copy())
print(f'yours| upper-triangular and  orthogonal test error ---> {np.linalg.norm(np.tril(my_R,-1)) + np.linalg.norm(np.eye(m) - my_Q @ my_Q.T)}')
print(f'yours| correctness test error ---> {np.linalg.norm(A - my_Q @ my_R)}')

# print
print(f'A = \n{np.around(A, 3)}')
print(f'Q = \n{np.around(Q, 3)}')
print(f'my_Q = \n{np.around(my_Q, 3)}')
print(f'R = \n{np.around(R, 3)}')
print(f'my_R = \n{np.around(my_R, 3)}')

numpy| upper-triangular and  orthogonal test error ---> 1.1312600865933311e-15
numpy| correctness test error ---> 2.7090938313991727e-15
yours| upper-triangular and  orthogonal test error ---> 1.9836923152161188e-15
yours| correctness test error ---> 2.675049346249148e-15
A = 
[[-0.51  -0.852 -0.81   1.398 -1.62  -0.043]
 [ 1.274  0.367 -0.108  0.219  0.847  1.303]
 [ 1.853  0.138  0.306  1.672 -2.25   0.044]
 [-0.282 -0.202 -0.636  0.612 -0.809  0.326]
 [ 2.076 -0.681  0.981  0.443  0.52   0.898]
 [ 0.894  1.867  0.763 -1.114  1.322  0.355]
 [-1.638 -1.604 -0.641 -1.611  0.288  0.804]
 [ 1.     1.329  0.871 -0.72  -0.348 -2.135]]
Q = 
[[-0.135 -0.246 -0.409  0.269 -0.125  0.114 -0.449  0.672]
 [ 0.338 -0.045 -0.524 -0.616  0.269  0.297  0.226  0.131]
 [ 0.492 -0.214 -0.272  0.183 -0.573 -0.408  0.328 -0.036]
 [-0.075 -0.035 -0.447 -0.077 -0.13  -0.023 -0.568 -0.669]
 [ 0.551 -0.552  0.43  -0.068  0.221  0.014 -0.39  -0.024]
 [ 0.237  0.571  0.057 -0.293  0.042 -0.564 -0.371  0.265]
 [

**Your answer: ```TODO ```**

In [13]:
# time and correctness test
numpy_test = {'time': 0.0, 'upper-triangular and  orthogonal test error':0.0, 'correctness test error':0.0}
yours_test = {'time': 0.0, 'upper-triangular and  orthogonal test error':0.0, 'correctness test error':0.0}

num_test = 100
for i in range(num_test):
    m, n = np.random.randint(40, 50), np.random.randint(40, 50)
    A = np.random.randn(m,n)

    # numpy code
    start_time = time.time()
    Q, R = np.linalg.qr(A.copy(), mode='complete')
    
    numpy_test['time'] += time.time() - start_time
    numpy_test['upper-triangular and  orthogonal test error'] += np.linalg.norm(np.tril(R,-1)) + np.linalg.norm(np.eye(m) - Q @ Q.T)
    numpy_test['correctness test error'] += np.linalg.norm(A - Q @ R)

    # your code
    start_time = time.time()
    my_Q, my_R = my_qr(A.copy())
    
    yours_test['time'] += time.time() - start_time
    yours_test['upper-triangular and  orthogonal test error'] += np.linalg.norm(np.tril(my_R,-1)) + np.linalg.norm(np.eye(m) - my_Q @ my_Q.T)
    yours_test['correctness test error'] += np.linalg.norm(A - my_Q @ my_R)

for name in numpy_test:
    numpy_test[name] /= num_test
    yours_test[name] /= num_test

yours_check = {'time': yours_test['time'] <= 0.01, 
               'upper-triangular and  orthogonal test error': yours_test['upper-triangular and  orthogonal test error'] <= 1e-5, 
               'correctness test error': yours_test['correctness test error'] <= 1e-5}

print(f'numpy| test ---> {numpy_test}')
print(f'yours| test ---> {yours_test}')
print(f'yours| check ---> {yours_check}')

numpy| test ---> {'time': 0.00024236679077148436, 'upper-triangular and  orthogonal test error': 4.095561311564944e-15, 'correctness test error': 2.1823452958255083e-14}
yours| test ---> {'time': 0.003951389789581299, 'upper-triangular and  orthogonal test error': 1.201453492491518e-14, 'correctness test error': 3.265587738886084e-14}
yours| check ---> {'time': True, 'upper-triangular and  orthogonal test error': True, 'correctness test error': True}


---

# B. Dictionary Learning

In [14]:
# # The allowed libraries
# from sklearn.feature_extraction.image import extract_patches_2d
# import numpy as np
# import matplotlib.pyplot as plt
# import imageio

In [15]:
# '''
# Download the necessary files from Google Drive via gdown library (source: https://pypi.org/project/gdown/) 

# Do the following steps:

# 1) Install gdown (if you are using Google Colab, skip this step). 
# In jupyter notebook: run (!pip install gdown) and (!pip install --upgrade gdown)

# 2) Run the following script to download files:
# '''

# # EE.png
# !gdown 12ONag930wZvpmkVQ0JV35qDVFjm9bq4J

# # EE_noisy.png
# !gdown 1FH8SYf_IaOWX0fu_buFIfwshrnUUC6CW

# # random_dictionary_64_121.npy
# !gdown 195AygBnFlzdBO5ikyc3lkCrxdKK8sSf3

In [16]:
# image_clean = imageio.imread('EE.png')[:, :].astype('float32')
# image_noisy = imageio.imread('EE_noisy.png')[:, :].astype('float32')

# plt.figure(figsize=(10,5))
# plt.subplot(1, 2, 1)
# plt.title('clean image')
# plt.imshow(image_clean, cmap='gray')
# plt.subplot(1, 2, 2)
# plt.title('noisy image')
# plt.imshow(image_noisy, cmap='gray')
# plt.show()

<div dir="rtl"> <font size="4" face="HM XNiloofar">
در این بخش از تمرین با استفاده از یادگیری واژه‌نامه، سیستمی به منظور نویززدایی تصاویر مقیاس خاکستری توسعه خواهید داد [1]. 
</font> </div>

[1] Elad, M. and Aharon, M., 2006. Image denoising via sparse and redundant representations over learned dictionaries. IEEE Transactions on Image processing, 15(12), pp.3736-3745.

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <br />
تصویر مقیاس خاکستری Y را می‌توان به صورت یک ماتریس r×c در نظر گرفت که هر مولفه آن عدد صحیحی در بازه [0,255] اختیار می‌نماید. یکی از نویزهای رایج در تصاویر، نویز گاوسی جمع‌شونده است که در نتیجه آن تصویر نویزی Y_n=Y+N در اختیار شما قرار می‌گیرد که N ماتریس نویز است. در سیستم نویززدایی مورد نظر در این تمرین، فرض می‌شود که قطعات هم‌اندازه در تصویر Y با ابعاد p×p از یک مدل تنک تولید شده‌اند. از آنجایی که تصویر اصلی در اختیار شما قرار ندارد، باید با استفاده از قطعات تصویر در Y_n، واژه‌نامه‌ای که قطعات تصویر اصلی از آن تولید شده‌اند را تخمین بزنید. سپس از این واژه‌نامه برای نویززدایی قطعات تصویر Y_n استفاده کنید. به این منظور در ادامه بخش‌های مختلف سیستم را پیاده‌سازی می‌نمایید و در انتها نیز با کنار هم قرار دادن این بخش‌ها، سیستم نهایی پیاده‌سازی خواهد گردید.
</font> </div>

## 1. Create dataset

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <b>
    ساخت پایگاه داده یادگیری واژه‌نامه:
    </b>
    <br />
        در این گام باید تابع <code>create_dataset</code> را پیاده‌سازی نمایید. ورودی‌های این تابع عبارتند از: 
<ul>
  <li> <code>image_noisy</code>:	تصویر نویزی با ابعاد r×c که هر مولفه آن عدد صحیحی در بازه [0,255] است و نوع آن <code>float32</code> است.</li>
  <li> <code>p</code>: طول قطعات تصویر.</li>
</ul>
 خروجی این تابع عبارتست از:
<ul>
<li><code>dataset</code>: 	پایگاه داده که یک ماتریس با ابعاد p^2×((r-p+1)(c-p+1)) است. هر ستون یکی از قطعات تصویر است که برداری شده است. از آنجایی که سایز تصویر r×c است و طول قطعات برابر با p است، در نتیجه در کل تصویر نویزی (r-p+1)(c-p+1) قطعه خواهیم داشت. هر قطعه نیز یک ماتریس مربعی p×p  است که با برداری‌سازی آن، برداری به طول p^2 خواهیم داشت . </li>
</ul>
 به منظور درستی پیاده‌سازی، به نکات زیر توجه نمایید.
 <ul>
  <li>	قطعه اول تصویر زیر ماتریس <code>image_noisy[0:8,0:8]</code> و قطعه دوم <code>image_noisy[0:8,1:9]</code> می‌باشد. یعنی قطعات به صورت سطری جلو می‌روند. </li>
  <li>	برای برداری‌سازی قطعات تصویر، سطرها را به صورت بردار عمودی در آورده و به ترتیب زیر هم قرار دهید تا یکی از ستون‌های ماتریس <code>dataset</code> را بسازد.</li>
     <li> 	برای پیاده‌سازی بهینه از تابع <code>extract_patches_2d</code> بهره ببرید. </li>
</ul>
</font> </div>

In [17]:
# def create_dataset(image_noisy, p):
    
#     # TODO
    
#     return dataset

## 2. Initialize dictionary

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <b>
   مقداردهی اولیه واژه‌نامه: 
    </b>
    <br />
مقداردهی اولیه واژه‌نامه از مراحلی است که می‌تواند کیفیت نویززدایی را تحت تاثیر قرار دهد. این واژه‌نامه می‌تواند با استفاده از DCT فوق کامل مقداردهی اولیه شود. در این تمرین به منظور ساده‌سازی همراه با سوالات تمرین یک واژه‌نامه نیز در اختیار شما قرار گرفته است که می‌توانید برای تست کار خود از آن استفاده نمایید. شما کافیست تابعی بنویسید که با دریافت آدرس واژه‌نامه، آن را لود کرده و در متغیر <code>init_dictionary</code> قرار داده و به خروجی بدهد. بنابراین ورودی این تابع عبارتست از:   
<ul>
  <li><code>address</code>:		آدرس واژه‌نامه اولیه </li>
</ul>
 خروجی این تابع عبارتست از:
<ul>
<li><code>init_dictionary</code>: 		واژه‌نامه اولیه </li>
</ul>
</font> </div>

In [18]:
# def create_init_dictionary(address):
    
#     # TODO
    
#     return init_dictionary

## 3. Learn dictionary

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <b>
  یادگیری واژه‌نامه: 
    </b>
    <br />در این بخش شما می‌بایست تابع <code>train_dictionary</code> را پیاده‌سازی نمایید. در گام نمایش تنک برای یک بردار دلخواه y_i از مجموعه آموزش می‌بایست نمایش تنک x_i را با حل مساله زیر به دست آورید:
 </font> </div>   
    
$$ x_i= \text{argmin}_x ‖x‖_0    \text{ subject to }   ‖y_i-Dx‖_2^2/l≤(\text{sigma}×\text{noise_gain})^2  $$

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <br />
که در رابطه فوق D ماتریس واژه‌نامه، <code>sigma</code> تخمین واریانس نویز گاوسی جمع‌شوند، <code>noise_gain</code> بهره نویز و l طول بردار y_i است.
<br />   
در این تابع برای گام نمایش تنک و به‌روزرسانی واژه‌نامه می‌بایست روش‌های مختلفی را پیاده‌سازی نمایید. برای گام نمایش تنک می‌بایست روش‌های زیر را پیاده‌سازی نمایید:
<ul>
  <li><code>MP</code>:	روش جستجوی انطابق </li>
  <li><code>OMP</code>:	روش جستجوی انطابق متعامد </li>
  <li><code>BOMP</code>: روش جستجوی انطابق متعامد دسته‌ای [2]</li>
</ul>
    برای گام به‌روزرسانی واژه‌نامه نیز می‌بایست دور روش زیر پیاده‌سازی شوند:
<ul>
  <li><code>AKSVD</code>:		تقریب الگوریتم [2] KSVD  </li>
  <li><code>MOD</code>:		الگوریتم MOD </li>
</ul>  
    توجه: در الگوریتم <code>MOD</code> امکان دارد ماتریس XX^T معکوس‌پذیر نباشد. به منظور رفع این مشکل از تنظیم کننده تیخونوف استفاده نمایید و به جای معکوس‌گیری از XX^T  از ماتریس I+XX^T  معکوس بگیرید.
<br />     
    بنابراین ورودی‌های این تابع عبارتست از:
<ul>
  <li><code>init_dictionary</code>:	واژه‌نامه اولیه </li>
  <li><code>dataset</code>:	پایگاه داده </li>
  <li><code>iter_num</code>:	تعداد تکرارهای یادگیری واژه‌نامه </li>
  <li><code>sigma</code>:	تخمین واریانس نویز </li>
  <li><code>noise_gain</code>:	بهره نویز </li>
  <li><code>SC_method</code>:	روش مورد استفاده برای نمایش تنک </li>
  <li><code>DL_method</code>:	روش مورد استفاده برای به‌روزرسانی واژه‌نامه </li>
</ul>
 خروجی‌های این تابع عبارتست از:
<ul>
  <li><code>final_dictionary</code>:		واژه‌نامه آموزش داده شده </li>
  <li><code>X</code>:		نمایش قطعات تصویر پایگاه داده در واژه‌نامه آموزش داده شده 
</ul>    
</font> </div>    
    
[2] Rubinstein, R., Zibulevsky, M. and Elad, M., 2008. Efficient implementation of the K-SVD algorithm using batch orthogonal matching pursuit (No. CS Technion report CS-2008-08). Computer Science Department, Technion.

In [19]:
# def train_dictionary(init_dictionary, dataset, iter_num, sigma, noise_gain, SC_method, DL_method):
#     D = init_dictionary
#     Y = dataset
#     for iter in range(iter_num):
#         print('iter %d'%iter)
#         X = sparse_coding(D, Y, tol=sigma * noise_gain, SC_method=SC_method)
#         D, X = dictionary_update(D, Y, X, DL_method=DL_method)
#     final_dictionary = D
#     return (final_dictionary, X)


# def sparse_coding(D, Y, tol, SC_method):
#     if SC_method.upper() == 'MP':
#         X = MP(D, Y, tol)
#     elif SC_method.upper() == 'OMP':
#         X = OMP(D, Y, tol)
#     elif SC_method.upper() == 'BOMP':
#         X = BOMP(D, Y, tol)
#     return X


# def dictionary_update(D, Y, X, DL_method):
#     if DL_method.upper() == 'AKSVD':
#         D, X = AKSVD(D, Y, X)
#     elif DL_method.upper() == 'MOD':
#         D, X = MOD(D, Y, X)
#     return (D, X)

# def MP(D, Y, tol):
    
#     # TODO
    
#     return X


# def OMP(D, Y, tol):
    
#     # TODO

#     return X


# def BOMP(D, Y, tol):
    
#     # TODO
    
#     return X


# def AKSVD(D, Y, X):
    
#     # TODO
    
#     return (D, X)


# def MOD(D, Y, X):
    
#     # TODO
    
#     return (D, X)



## 4. Reconstruct image

<div dir="rtl"> <font size="4" face="HM XNiloofar">
    <b>
بازسازی تصویر: 
    </b>
    <br />
    در قسمت قبل شما توانستید نمایش تمام قطعات تصویر در واژه‌نامه نهایی (<code>final_dictionary</code>) را به دست آورید. در این قسمت به‌جای هر قطعه از تصویر، بازسازی شده آن را جایگزین می‌نمایید. فرض کنید x_i نمایش یک قطعه برداری شده از تصویر در واژه‌نامه نهایی (D_f) باشد. در این صورت تخمین بدون نویز از آن قطعه برداری شده را می‌توان به صورت D_f x_i ساخت و با استفاده از عملیات معکوس برداری‌سازی (تبدیل بردارها به قطعات تصویر)، می‌توان تخمینی از قطعه تصویر بدون نویز به دست آورد. از آنجایی که شما تمام قطعات قابل استخراج از تصویر را در قسمت الف در نظر گرفتید، بنابراین هر پیکسل از تصویر در تعدادی از قطعات قرار می‌گیرد (پیکسل‌های مرکزی دقیقا در p^2 قطعه و پیکسل‌های حاشیه‌ای در کمتر از این مقدار قطعه تصویر حضور دارند). به منظور نویززدایی بهتر می‌توان متوسط تمام تخمین‌ها از یک پیکسل را در نظر گرفت. به عبارت دیگر یک ماتریس O و یک ماتریس C صفر در نظر بگیرید. حال هر یک از قطعات نویزدایی شده را با بخشی از ماتریس O که متناظر با آن است جمع نمایید. همزمان به بخش متناظر با قطعه تصویر در ماتریس C نیز یک واحد اضافه کنید. پس از قرار دادن تمام قطعات نویز زدایی شده، اگر ماتریس O را به صورت عنصر به عنصر بر ماتریس C تقسیم نمایید، خروجی مورد نظر به دست می‌آید. 
    <br />
    روال فوق معمولا منجر به بایاس در تصویر حاصل می‌گردد. به منظور کاهش این بایاس نیز ضریبی از تصویر نویزی با خروجی حاصل از الگوریتم فوق ترکیب می‌شود. فرض کنید طبق روال فوق، O و C را به دست آورده‌اید. خروجی نهایی از رابطه زیر به دست خواهد آمد:
    <center><code>image_denoised =  (O + m × image_noisy) / (C + m)</code></center>
در این گام شما می‌بایست تابع بازسازی تصویر با نام <code>reconstruct_image</code> را پیاده‌سازی نمایید. این تابع ورودی‌های زیر را می‌پذیرد:
<ul>
  <li><code>final_dictionary</code>:	واژه‌نامه آموزش داده شده </li>
  <li><code>X</code>:   	نمایش قطعات تصویر پایگاه داده در واژه‌نامه آموزش داده شده </li>
  <li><code>image_noisy</code>:   	تصویر نویزی با ابعاد r×c که هر مولفه آن عددی صحیحی در بازه [0,255] است و نوع آن <code>float32</code> است. </li>
  <li><code>p</code>:  	طول قطعات تصویر</li>
   <li><code>m</code>:  	ثابت ترکیب تصویر نویزی</li>
</ul>
 خروجی این تابع عبارتست از:
<ul>
<li><code>image_denoised</code>:	تصویر نویززدایی شده با ابعاد r×c که هر مولفه آن عددی صحیحی در بازه [0,255] است و نوع آن <code>float32</code> است.</li>
</ul>
</font> </div>

In [20]:
# def reconstruct_image(final_dictionary, X, image_noisy, p, m):
    
#     # TODO
    
#     return image_denoised

## 5. Evaluation

<div dir="rtl"> <font size="4" face="HM XNiloofar">
حال  تمام ماژول‌های مورد نیاز برای ساخت سیستم نویززدایی را در اختیار دارید. تابع <code>DL_denoising</code> سیستم‌ فوق را یکپارچه می‌نماید و خروجی شما را می‌سازد.
<br />    
به منظور اینکه بتوانید پیاده‌سازی خود را چک نمایید، یک تصویر نویز پاک (<code>EE.png</code>) یک نمونه نویزی شده آن با انحراف معیار 20 (<code>EE_noisy.png</code>) و یک دیکشنری اولیه تصادفی (<code>random_dictionary_64_121.npy</code>) در اختیار شما قرار گرفته است. نتایج حاصل از نویززدایی این تصویر برای حالت‌های مختلف مختلف مشاهده کنید. در هر حالت ماتریس <code>D</code> و <code>X</code> خروجی را در Google Drive آپلود کنید. فایل‌ها را در حالت دسترسی عمومی قرار دهید. ID لینک هر فایل را در محل مربوطه بنویسید. با اجرای cell مربوطه باید هر شخصی بتواند فایل‌ها را دانلود و نتیجه نهایی را بازتولید کند.
 </font> </div>   

In [21]:
# def DL_denoising(address_image_noisy, address_init_dictionary, p, iter_num, sigma, noise_gain, SC_method, dl_method, m):
#     image_noisy = imageio.imread(address_image_noisy)[:, :].astype('float32')
#     dataset = create_dataset(image_noisy, p)
#     D = create_init_dictionary(address_init_dictionary)
#     D, X = train_dictionary(D, dataset, iter_num, sigma, noise_gain, SC_method, dl_method)
#     image_denoised = reconstruct_image(D, X, image_noisy, p, m)
#     return (image_denoised, image_noisy, D, X)

# def Calculate_PSNR(image1, image2):
#     mse = np.sum((image1-image2)**2)/np.prod(image1.shape)
#     return 10*np.log10((255**2)/mse)

In [22]:
# image_denoised, image_noisy, D_MP_MOD, X_MP_MOD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='MP', dl_method='MOD', m=0.5)
# np.save('D_MP_MOD', D_MP_MOD)
# np.save('X_MP_MOD', X_MP_MOD)

# image_denoised, image_noisy, D_OMP_MOD, X_OMP_MOD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='OMP', dl_method='MOD', m=0.5)
# np.save('D_OMP_MOD', D_OMP_MOD)
# np.save('X_OMP_MOD', X_OMP_MOD)

# image_denoised, image_noisy, D_BOMP_MOD, X_BOMP_MOD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='BOMP', dl_method='MOD', m=0.5)
# np.save('D_BOMP_MOD', D_BOMP_MOD)
# np.save('X_BOMP_MOD', X_BOMP_MOD)

# image_denoised, image_noisy, D_MP_AKSVD, X_MP_AKSVD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='MP', dl_method='AKSVD', m=0.5)
# np.save('D_MP_AKSVD', D_MP_AKSVD)
# np.save('X_MP_AKSVD', X_MP_AKSVD)

# image_denoised, image_noisy, D_OMP_AKSVD, X_OMP_AKSVD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='OMP', dl_method='AKSVD', m=0.5)
# np.save('D_OMP_AKSVD', D_OMP_AKSVD)
# np.save('X_OMP_AKSVD', X_OMP_AKSVD)

# image_denoised, image_noisy, D_BOMP_AKSVD, X_BOMP_AKSVD = DL_denoising('EE_noisy.png', 'random_dictionary_64_121.npy', p=8, iter_num=20, sigma=20, noise_gain=1.075, SC_method='BOMP', dl_method='AKSVD', m=0.5)
# np.save('D_BOMP_AKSVD', D_BOMP_AKSVD)
# np.save('X_BOMP_AKSVD', X_BOMP_AKSVD)

# # upload the saved files to Google Drive, then their access to 'Everyone with the link'. 


In [23]:
# plt.figure(figsize=(10,20))

# plt.subplot(4, 2, 1)
# plt.title('image_clean')
# plt.imshow(image_clean, cmap='gray')

# psnr = Calculate_PSNR(image_clean, image_noisy)
# print('\n image_noisy')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 2)
# plt.title('image_noisy ' + str(np.around(psnr, 3)))
# plt.imshow(image_noisy, cmap='gray')

# p = 8
# m = 0.5

# # --------------- MP_MOD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_MP_MOD.npy')
# X = np.load('X_MP_MOD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n MP+MOD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 3)
# plt.title('MP+MOD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')

# # --------------- OMP_MOD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_OMP_MOD.npy')
# X = np.load('X_OMP_MOD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n OMP+MOD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 4)
# plt.title('OMP+MOD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')

# # --------------- BOMP_MOD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_BOMP_MOD.npy')
# X = np.load('X_BOMP_MOD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n BOMP+MOD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 5)
# plt.title('BOMP+MOD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')


# # --------------- MP_AKSVD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_MP_AKSVD.npy')
# X = np.load('X_MP_AKSVD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n MP+AKSVD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 6)
# plt.title('MP+AKSVD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')

# # --------------- OMP_AKSVD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_OMP_AKSVD.npy')
# X = np.load('X_OMP_AKSVD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n OMP+AKSVD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 7)
# plt.title('OMP+AKSVD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')

# # --------------- BOMP_AKSVD --------------- #
# # download D
# !gdown # TODO
# # download X
# !gdown # TODO

# D = np.load('D_BOMP_AKSVD.npy')
# X = np.load('X_BOMP_AKSVD.npy')
# image_denoised = reconstruct_image(D, X, image_noisy, p, m)
# psnr = Calculate_PSNR(image_clean, image_denoised)
# print('\n BOMP+AKSVD')
# print(psnr)
# print(psnr >= 24)
# plt.subplot(4, 2, 8)
# plt.title('BOMP+AKSVD ' + str(np.around(psnr, 3)))
# plt.imshow(image_denoised, cmap='gray')

# plt.show()

---

# Give Us Feedback

In [24]:
# from ipywidgets import Label, IntSlider, Textarea, Button, Layout, HBox, VBox
# from IPython.display import clear_output

# IntSlider_list = {i:IntSlider(value=3, min=1, max=5) for i in range(5)}
# H1 = HBox([Label("How was the homework?")])
# H2 = HBox([Label("Please rate the homework from 1 to 5 (lowest to highest).")])
# H3 = HBox([Label("The assignment was instructive.", layout=Layout(width="22%")), IntSlider_list[0]])
# H4 = HBox([Label("It was not time consuming.", layout=Layout(width="22%")), IntSlider_list[1]])
# H5 = HBox([Label("The questions were clear.", layout=Layout(width="22%")), IntSlider_list[2]])
# H6 = HBox([Label("The homework was well organized.", layout=Layout(width="22%")), IntSlider_list[3]])
# H7 = HBox([Label("Finally, you had a good vibe!", layout=Layout(width="22%")), IntSlider_list[4]])
# textarea = Textarea(value='', placeholder='Any comments or suggestions', description='', disabled=False)
# button = Button(description='Submit', disabled=False, button_style='', tooltip='Click me', icon='paper-plane')
# H8 = HBox([textarea])
# H9 = HBox([button])

# def ff(button):
#     button.button_style = 'success'
#     score = sum([IntSlider_list[i].value for i in IntSlider_list]) / 5
#     x = round(score*4)/4
#     y = int(x) * '🌕' + ((x-int(x))==0 and not x==5) * '🌑' + ((x-int(x))==0.25) * '🌘' + ((x-int(x))==0.5) * '🌗' + ((x-int(x))==0.75) * '🌖' + (4-int(x)) * '🌑'
#     clear_output(wait=True)
#     display(VBox([H1, H2, H3, H4, H5, H6, H7, H8, H9]))
#     print("Thank you for your feedback! 😊")
#     print(f"Score: {y}")
#     print("\nFeedback:",*[IntSlider_list[i].value for i in IntSlider_list])
#     print(textarea.value)

# button.on_click(ff)

# VBox([H1, H2, H3, H4, H5, H6, H7, H8, H9])

In [25]:
# # To export jupyter notebook to html, save your notebook and run this cell
# !jupyter nbconvert --to html PHW.ipynb